In [2]:
#!jupyter nbconvert --to=python F2R_sensitivity_toolbox.ipynb

[NbConvertApp] Converting notebook F2R_sensitivity_toolbox.ipynb to python
[NbConvertApp] Writing 60442 bytes to F2R_sensitivity_toolbox.py


# plot mesh overview

In [ ]:
# plot mesh overview 
def check_plot_mesh(mesh, plot_globe = False):
    '''check mesh setup nodal and area
    input: pathto/fesom.mesh.diag.nc
    
    plot_globe: get overview plots of mesh structures
    '''
    import pyfesom2 as pf
    import numpy as np
    
    #runid = 'mo4/fesom'
    #meshdiag=resultpath+'/'+runid+'.mesh.diag.nc'
    #!ncdump -h $meshdiag

    diag = pf.get_meshdiag(mesh)
    #print(diag)
    #print(diag['Z']) # depth of layers
    mesh_depths = diag['Z'].values
    print('Mesh layer depths:'), print(mesh_depths)
    
    nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
    nod_area.load()
    #print(np.shape(nod_area[:,:]), type(nod_area))
    
    if plot_globe:    
        # whole globe
        pf.plot(mesh,np.array(nod_area[0,:]),units='Surface nodal area')

        # plot nodal area and mesh structure together
        pf.tplot(mesh, np.array(nod_area[0,:]), ptype='tri', box=[-30, 30, 60, 82], mapproj='np',lw=0.5,units='nodal area')
        pf.tplot(mesh, np.array(nod_area[0,:]), ptype='tri', box=[-30, 30, -30, 30], mapproj='merc',lw=0.5,units='nodal area')

# Draw cartopy natural earth feature to axis

In [ ]:
def mygrid(m,grid=False,coast=True):
    '''
    Draw cartopy natural earth feature to axis m
    
    if grid == True: returns grid instance g
    further modification to grid:
    e.g. g.top_labels = False, g.left_labels = False, g.xlines = False
    '''
    import cartopy.feature as cfeature
    
    if(coast == True):
        #m.coastlines(resolution='110m', color='black', linewidth=1)
        land_poly = cfeature.NaturalEarthFeature('physical', 'land', '110m',edgecolor='k',facecolor=cfeature.COLORS['land'])
        m.add_feature(land_poly)

    if(grid == True):
        g1 = m.gridlines(draw_labels = True)
        g1.xlabels_top = False
        g1.xlabel_style = {'size': 16, 'color': 'gray'}
        g1.ylabel_style = {'size': 16, 'color': 'gray'}
        return g1
 
## old version
#def mygrid(m,coast = True):
#    g1 = m.gridlines(draw_labels = True)
#    g1.xlabels_top = False
#    g1.ylabels_right = False
#    g1.xlabel_style = {'size': 12, 'color': 'gray'}
#    g1.ylabel_style = {'size': 12, 'color': 'gray'}
#    
#    if(coast == True):
#        m.coastlines(resolution='110m', color='black', linewidth=1)
#        #m.add_feature(land_poly)



# plot Taylor normalized

In [ ]:
def plt_Taylor_norm(data_ref,data_pred,
                mask=True,
                title='Normalized Taylor Diagram',
                label=['Observation', 'Model'],
                loc='right',
                tickRMSangle = 135.0,
                verbose = True,
                plot = True):
    '''
    Reference: observation
    Predicted: model data
    
    careful: here used different order as in input in skill_metrics: sm.taylor_statistics(pred,ref)
    
    Use this function to get Taylor statistics and plot them as normalized to std(reference) = 1
    
    Root mean sqare error are centered, see sm.centered_rms_dev? for details
    
    Applied normalization: 
        std_normalized(ref) = std(ref) / std(ref)
        std_normalized(pred) = std(pred) / std(ref)
        ... similar for CRMSD
        
    Input:
    data_ref: array with references/observation data
    data_pred: array with model data
    mask: bolean if observation data should be masked with np.where(data_pred !=0)
    title: string for plot title
    label: list with 2 entries for label for 'Observation', 'Model'
    loc: string, positional argument for legend
    tickRMSangle: used for ticks of RMSE in Taylor plot
    verbose: print calculated Taylor statistics
    plot: plot Taylor diagram
    
    Output:
    fig, sdev, crmsd, ccoef
    
    Changed style of marker in skill_metrics/plot_pattern_diagram_markers.py
    '''
    import numpy as np
    import matplotlib.pyplot as plt
    import skill_metrics as sm
                    
    if mask == True:
        # get statistics only from ocean gridpoints (where model data != 0)
        ind_stat = np.where(data_pred != 0)
        taylor_stats1 = sm.taylor_statistics(data_pred[ind_stat],data_ref[ind_stat])
    else:
        taylor_stats1 = sm.taylor_statistics(data_pred,data_ref)


    # original
    sdev = np.array([taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]])
    
    crmsd = np.array([taylor_stats1['crmsd'][0],
                      taylor_stats1['crmsd'][1]])
    
    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    if verbose:
        print('\nOriginal Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
            sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))
    
    # normalized    
    sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])
    
    crmsd = np.array([taylor_stats1['crmsd'][0]/taylor_stats1['sdev'][0],
                      taylor_stats1['crmsd'][1]/taylor_stats1['sdev'][0]])
    
    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    if verbose:
        print('\nNormalized Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
            sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))

    if plot:
        if(sdev[1] > 1):
            axismax = np.round(sdev[1],0) +0.5
            print("Adjust axis max to {0}".format(axismax))
            tickSTD = np.arange(0,axismax,0.25)
        else: 
            axismax = 1.25
            tickSTD = [0,0.25,0.5,0.75,1.25]
    
        fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
        sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                              titleOBS = 'observation', markerLabel = label,
                              markerLabelColor = 'c',
                              markerColor = 'c', markerLegend = 'on',
                              tickRMS = [0,0.25,0.5,0.75,1.25],
                              tickRMSangle = tickRMSangle,
                              colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                              titleRMS = 'off', tickSTD = tickSTD,# tickSTD = [0,0.25,0.5,0.75,1.25],
                              axismax = axismax, #axismax = 1.25, 
                              colSTD = 'b', styleSTD = '-.',
                              widthSTD = 1.0, titleSTD = 'on',
                              colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                              titleCOR = 'on')

        plt.title(title, loc=loc)   
    
    else:
        fig = 'none'

    return fig, sdev, crmsd, ccoef

# load WOA to use as ocean mask for 1x1 interpolated FESOM data

In [ ]:
### load WOA to use as ocean mask for 1x1 interpolated FESOM data

def get_seamask_WOA(    
    ncfile = '/work/ollie/projects/MarESys/evaluation/woa13_all_n00_01.nc',
    ncvariable ='n_an', # n_an:standard_name = "moles_concentration_of_nitrate_in_sea_water"
    return_3D = False,
    get_overview = False):
    
    '''
    Load WOA example data to extract sea mask for application on 1x1 grid.
    
    Return:
    mask = [0;1] for ocean, land
    X360, Y180 = WOA grid
    DepthRaw = WOA depth if return_3D = True
    
    
    get_seamask_WOA(
        ncfile = /work/ollie/projects/MarESys/evaluation/woa13_all_n00_01.nc
        ncvariable ='n_an'
        return_3D = False,
        get_overview = False
    )
    '''
    
    import numpy as np
    from netCDF4 import Dataset
    #from scipy.interpolate import griddata
    
    # load NetCDF 
    print('***\nLoading WOA file: {0}'.format(ncfile))
    f          = Dataset(ncfile, 'r')
    DepthRaw   = -f.variables['depth'][:]                                
    lonwoa     =  f.variables['lon'][:]
    latwoa     =  f.variables['lat'][:]
    Timewoa     =  f.variables['time'][:]
    VARwoa     =  f.variables[ncvariable][:]
    VARwoa     = np.squeeze(VARwoa)
    VARwoa     = np.ma.filled(VARwoa, np.nan)

    X360, Y180 = np.meshgrid(lonwoa, latwoa)

    if(get_overview==True):
        #!ncdump -h $ncfile
        import matplotlib.pyplot as plt

        fig = plt.figure(figsize= (7,7))
        ax = plt.subplot()
        im = ax.pcolor(X360, Y180, VARwoa[0,:,:])
        cbar = fig.colorbar(im, orientation = 'horizontal')
        cbar.set_label(ncvariable) 
        plt.title('WOA var "{0}"'.format(ncvariable))
        
    mask = np.copy(VARwoa)
    mask[np.isfinite(VARwoa)] = 0
    mask[np.isnan(VARwoa)] = 1
    
    mask = np.ma.filled(mask, fill_value = 0)
        
    if(get_overview == True):
        fig = plt.figure(figsize= (7,7))
        ax = plt.subplot()
        im = ax.pcolor(X360, Y180, mask[0,:,:])
        cbar = fig.colorbar(im, orientation = 'horizontal')
        cbar.set_label('sea mask derived from WOA') 

    if(return_3D == False):
        mask = mask[0,:,:]
        print('Returning 2D mask on original WOA grid X360, Y180\nwith shape {}\n***'.format(
            np.shape(mask)))
        return mask, X360, Y180
        
    else:
        print('Returning 3D mask on original WOA grid DepthRaw, X360, Y180\nwith shape {}\n***'.format(
            np.shape(mask)))
        return mask, X360, Y180, DepthRaw
        
    

In [ ]:
if __name__ == "__main__":
    import numpy as np
    
    mask, x, y = get_seamask_WOA(get_overview=True)
    
    print(mask)
    print(np.shape(mask),type(mask))

# Load FESOM monthly data for seasonal cycle

In [ ]:
class var_monthly:   
    '''
    class var_monthly(resultpath, var_name, years, mesh, silent=False, only2D=False)
    
    Input:
    resultpath: path to FESOM-REcoM model output
    var_name: variable name used in netcdf4 model output
    years: range of years to analyze
    mesh: pyfesom2 mesh object
    silent: print diagnostics of processings
    only2D: set True if input variable has only 2D shape
    
    Output:
    data_monthly: monthly model data with shape: (len of years, 12 months, mesh.x2, mesh.zlev)

    '''
    def __init__(self,resultpath, var_name, years, mesh, silent=False, only2D=False):
        
        import numpy as np
        import pyfesom2 as pf
        import matplotlib.pyplot as plt
        #from netCDF4 import Dataset
        #import cartopy.crs as ccrs
    
        self.resultpath = resultpath
        self.var_name = var_name
        self.years = years
        self.mesh = mesh
        self.silent = silent
        self.only2D = only2D

        self.months = np.arange(1,13,1)
        
        if only2D:
            var_monthly_aux = np.zeros((len(self.years),len(self.months),len(self.mesh.x2)))
            if not self.silent:
                print('Processing 2D for variable "{}"'.format(self.var_name))

        else:
            var_monthly_aux = np.zeros((len(self.years),len(self.months),len(self.mesh.x2),len(self.mesh.zlev)-1))
            if not self.silent:
                print('Processing all depths for variable "{}"'.format(self.var_name))

        # load data for all depth 
        for i in range(len(self.years)):
            ind_year = int(self.years[i])

            if not silent:
                print('loading {0}'.format(ind_year))

            var_raw = pf.get_data(self.resultpath, self.var_name, ind_year, self.mesh, 
                                  how="ori", compute=False, silent=True)

            for m in range(len(self.months)):
                if only2D:
                    var_monthly_aux[i,m,:] = var_raw[m,:]
                else:
                    var_monthly_aux[i,m,:,:] = var_raw[m,:,:]

        if not silent:
            print('Processed {0} years of input data\nResulting shape: {1}'.format(
                len(self.years),np.shape(var_monthly_aux)))

        self.data_monthly = var_monthly_aux


    def seasonalcycle(self):
        '''
        Calculate monthly average over years, i.e. seasonal cycle.
        Caculcated per depth level.
        '''
        
        import numpy as np
        
        mean_monthly_alldepths = np.nanmean(self.data_monthly,axis=0)
        print('\nSeasonal cycle over years {1} to {2} \nResulting shape {0}'.format(
            np.shape(mean_monthly_alldepths), self.years[0], self.years[-1]))
        
        self.data_seasonalcycle = mean_monthly_alldepths
        
        
    
    def volwmean(self, use_seasonal = True, depthrange=[0,-100]):
        '''
        Volume-weighted mean over specified depth range.
        
        Depth range can be specified, default: depthrange=[0,-100]
        '''
        
        import numpy as np
        import pyfesom2 as pf
        
        self.use_seasonal = use_seasonal
        self.depthrange = depthrange
        
        ### retrieving depth indices
        i_depth0 = pf.ind_for_depth(self.depthrange[0], self.mesh)
        i_depth1 = pf.ind_for_depth(self.depthrange[1], self.mesh)
        
        if not self.silent:
            print('\nVolume-weighted mean:\nUsing depth range of {0} to {1} m, i.e. indices {2} to {3}'.format(
                self.depthrange[0], self.depthrange[1], i_depth0, i_depth1))
            
        ### load FESOM diag 
        meshdiag = self.resultpath+'/fesom.mesh.diag.nc'
        diag = pf.get_meshdiag(self.mesh,meshdiag=meshdiag) 

        ### layer depth  
        # c.f. Danilov et al. (2017)
        # "in the vertical direction, the horizontal velocities and scalars are located at mid-levels" 
        layer_depths = -diag['Z'].values # depth of layers
        layer_depths = layer_depths[:-1] # lowest depth seems to be benthic?!

        # --> layer thickness
        delta_z = np.abs(np.diff(self.mesh.zlev))
        delta_aux = np.tile(delta_z,(126858,1))
        
        ### load nodal area 
        nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
        nod_area.load()
        nod_area = np.swapaxes(np.array(nod_area.data),0,1)

        ### nodal volume
        nod_vol = nod_area[:,:-1] * delta_aux            
         
        ### processing data
        if self.use_seasonal:
            data = self.data_seasonalcycle
            nod_vol = np.tile(nod_vol,(12,1,1))
            
        else:
            data = self.data_monthly
            #nod_vol = np.tile(nod_vol, ())
            print('to be implemented')
        
        ### applying depth range
        data = np.squeeze(data[:,:,i_depth0:i_depth1])
        nod_vol = np.squeeze(nod_vol[:,:,i_depth0:i_depth1])
        
        if not self.silent:
            print('Shape of data: {0}'.format(np.shape(data)))
            print('Shape of nod_vol: {0}'.format(np.shape(nod_vol)))
        
        aux_w_data = np.nansum(data * nod_vol, axis = 2)
        aux_tot_vol = np.nansum(nod_vol, axis = 2) 
            
        self.data_monthly_wm =  aux_w_data / aux_tot_vol
        self.nod_vol = nod_vol   
        
        if not self.silent:
            print('Shape of data_monthly_wm: {0}'.format(np.shape(self.data_monthly_wm)))
        

# Load WOA interpolated to FESOM grid

In [ ]:
class WOAdata:
    '''
    Load World Ocean Atlas NetCDF4 file and interpolate to FESOM mesh.
    
    Careful: Output still needs to be masked with an ocean bathymetry mask, e.g. with FESOM output!
    
    In: ncfile,ncvariable,meshpath
    
    Out: class WOAdata with self.woa_int containing interpolated WOA ncvariable
    '''
    
    def __init__(self,runname,resultpath,meshpath,ncfile,ncvariable,get_overview=False):
    
        self.runid = runname
        self.resultpath = resultpath
        self.meshpath = meshpath
        self.ncfile = ncfile
        self.ncvariable = ncvariable
        self.get_overview = get_overview

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        #import skill_metrics as sm
        #import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        # load NetCDF ------------------------------------------------------------------------------------
        print('***\nLoading WOA file: {0}\n***'.format(self.ncfile))
        f          = Dataset(self.ncfile, 'r')
        DepthRaw   = -f.variables['depth'][:]                                
        lonwoa     =  f.variables['lon'][:]
        latwoa     =  f.variables['lat'][:]
        Timewoa     =  f.variables['time'][:]
        VARwoa     =  f.variables[ncvariable][:]
        VARwoa     = np.squeeze(VARwoa)
        VARwoa     = np.ma.filled(VARwoa, np.nan)

        X360, Y180 = np.meshgrid(lonwoa, latwoa)

        if(self.get_overview==True):
            #!ncdump -h $self.ncfile
            
            fig = plt.figure(figsize= (7,7))
            ax = plt.subplot()
            im = ax.pcolor(X360, Y180, VARwoa[0,:,:])
            cbar = fig.colorbar(im, orientation = 'horizontal')
            cbar.set_label(ncvariable) 
            plt.title('WOA var "{0} before interpolation"'.format(self.ncvariable))

        # load FESOM mesh ------------------------------------------------------------------------------------
        mesh = pf.load_mesh(self.meshpath)
        
        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.resultpath+'/'+self.runid+'.mesh.diag.nc'
        #!ncdump -h $meshdiag

        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runid)
        #print(diag)
        #print(diag['Z']) # depth of layers
        mesh_depths = diag['Z'].values

        #nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
        #nod_area.load()
        #print(np.shape(nod_area[:,:]), type(nod_area))
        
        # check maximum depth in WOA compared to FESOM
        dmin_woa = np.min(DepthRaw)
        dmin_fesom = np.min(mesh_depths)#mesh.zlev)

        if(dmin_woa <= dmin_fesom):
            print('***\nDepth greater in WOA ({0}) than in FESOM ({1})'.format(dmin_woa, dmin_fesom))
            ilev = len(mesh_depths)
            max_zlev = mesh_depths[ilev-1]
        else:
            print('***\nDepth greater in FESOM ({1}) than in WOA ({0})'.format(dmin_woa, dmin_fesom))
            ilev = np.where(mesh_depths >= dmin_woa)
            ilev = ilev[0][-1]
            max_zlev = mesh_depths[ilev]

        print('Please consider choosing max depth level {0} with max depth at {1}!\n***'.format(ilev,max_zlev))
    

        # storage container
        woa_int = np.zeros((len(mesh.zlev)-1,len(mesh.x2)))
        #print(np.shape(din_int))

        for k in range(0,len(mesh_depths)): # depth are layer depth --> between levels, c.f. FESOM documentation
            lev = mesh_depths[k] # current FESOM depth
            ind1 = np.where(DepthRaw >= lev)
            ind1 = ind1[0][-1]
            ind2 = np.where(DepthRaw < lev)[0]

            if ind2.size > 0:                            # If we have not yet reached the bottom
                ind2 = ind2[0]                           # The index of the depth level below the current fesom level
                c    = DepthRaw[ind1]-DepthRaw[ind2]     # Difference in depth between the data value above and below the fesom depth
                c1   = DepthRaw[ind1]-lev                # Difference between fesom depth and data depth above
                c2   = -(DepthRaw[ind2]-lev)             # Difference between fesom depth and data depth below
                c1   = (c-c1)/c                          # Scaling coefficient for the depth above
                c2   = (c-c2)/c                          # Scaling coefficient for the depth below
            else:                                        # We have reached the bottom
                c1   = 1.
                c2   = 0.
                ind2 = ind1

            indZ  = np.where(mesh_depths == lev)                               
            # original code:
            # indZ  = np.where(-mesh.z3 == lev)          # Find the mesh index of the current fesom depth
            indZ = indZ[0] 
            if False: #(self.get_overview == True):
                print('\nFESOM depth = {0}, WOA depths = {1}, {2} \nDepth indices: {3} {4},  FESOM index: {5} \nScaling c1 = {6}, c2 = {7}'.format(lev,DepthRaw[ind1],DepthRaw[ind2],ind1, ind2,indZ,c1,c2))


            aux1  = VARwoa[ind1,:,:]                     # Find the data above the current fesom depth
            aux2  = VARwoa[ind2,:,:]                     # Find the data below the current fesom depth
            aux   = np.squeeze(c1*aux1+c2*aux2)          # Scaling the data according to vertical distribution as found above
            ind   = np.squeeze(~np.isnan(aux)) 
            #print(np.shape(aux), np.shape(ind))

            # first interpolation to original grid to close empty parts
            aux = griddata((X360[ind], Y180[ind]), aux[ind], (X360, Y180), method='nearest')                             
            # 2D field without nans                           

            # second interpolation to FESOM grid
            woa_int[indZ,:] = griddata((X360.ravel(), Y180.ravel()), aux.ravel(), 
                                   (mesh.x2, mesh.y2), method='nearest')  
            # Final interpolated field

            if np.isnan(np.min(woa_int)): print('WARNING: The interpolated field contains NaNs at depth',lev)                 # Testing if results contain NaNs. If yes, the routine needs adjustments

            if(self.get_overview ==True):
                print('Depth: {0} min = {1} max = {2} mean = {3}'.format(lev,np.min(woa_int), np.max(woa_int), np.mean(woa_int)))

        woa_int = np.swapaxes(woa_int,0,1) # adjust axes layout to FESOM output
        #print(np.shape(woa_int))    
        
        self.woa_int = woa_int

# Load MarEDAt

In [ ]:
def maredat_load(ncfileMaredat):
    '''
    Input: 
    ncfileMaredat
    
    Output:
    lon_maredat
    lat_maredat
    maredat_layered biomass in [mg C/m3]
    '''
    
    import numpy as np
    from netCDF4 import Dataset
    
    f          = Dataset(ncfileMaredat, 'r')
    d_biomass  = f['BIOMASS'][:] # mug C/L = mg C /m3
    lon_maredat      = f['LONGITUDE'][:]
    lat_maredat    = f['LATITUDE'][:]
    depth_maredat    = f['DEPTH'][:]

    d_biomass[:][d_biomass[:] == 1e+35] = np.nan # remove fill value
    #print(np.shape(d_biomass))

    # mean over year
    d_biomass_annual = np.nanmean(d_biomass[:],axis=0)
    #print(np.shape(d_biomass_annual))

    # depth indices
    dep_ind1  = np.nonzero((0 <= depth_maredat[:] )&(depth_maredat[:] <= 5))
    dep_ind2  = np.nonzero((5 < depth_maredat[:] )&(depth_maredat[:] <= 25))
    dep_ind3  = np.nonzero((25 < depth_maredat[:] )&(depth_maredat[:] <= 100))
    dep_ind4  = np.nonzero((100 < depth_maredat[:]))

    # mean over depth ranges,
    # fill masked array to normal array
    #d_biomass_tot = np.squeeze(np.nanmean(d_biomass_annual, axis=0))
    d_biomass_0_5 = np.squeeze(np.nanmean(d_biomass_annual[dep_ind1], axis=0))
    d_biomass_5_25 = np.squeeze(np.nanmean(d_biomass_annual[dep_ind2], axis=0))
    d_biomass_25_100 = np.squeeze(np.nanmean(d_biomass_annual[dep_ind3], axis=0))
    d_biomass_100_bot = np.squeeze(np.nanmean(d_biomass_annual[dep_ind4], axis=0))
    
    maredat_layered = [d_biomass_0_5,d_biomass_5_25,d_biomass_25_100,d_biomass_100_bot]

    if False: # old!
        # if you use this, first remove np.nanmean from all lines!
        # fill masked array to normal array
        #d_biomass_tot = np.ma.filled(np.squeeze(np.nanmean(d_biomass_annual, axis=0)),0)
        d_biomass_0_5 = np.ma.filled(np.squeeze(np.nanmean(d_biomass_annual[dep_ind1], axis=0)),0)
        d_biomass_5_25 = np.ma.filled(np.squeeze(np.nanmean(d_biomass_annual[dep_ind2], axis=0)),0)
        d_biomass_25_100 = np.ma.filled(np.squeeze(np.nanmean(d_biomass_annual[dep_ind3], axis=0)),0)
        d_biomass_100_bot = np.ma.filled(np.squeeze(np.nanmean(d_biomass_annual[dep_ind4], axis=0)),0)

    #print('Shape of maredat_layered[0]:')
    #print(np.shape(maredat_layered[0]))
    #print(np.shape(d_biomass_0_5))
    
    return lat_maredat, lon_maredat, maredat_layered

In [ ]:
def maredat_overview_plot(maredat_layered,lon_maredat, lat_maredat,mapproj = 'rob'):
    '''
    Input: 
    maredat_layered: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom
    lon_maredat
    lat_maredat
    mapproj = 'rob': mapprojection abbreviation as in pyfesom2
    
    Output:
    figure
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    if(mapproj == 'pc'): mapproj = ccrs.PlateCarree()
    elif(mapproj == 'rob'): mapproj = ccrs.Robinson()
    else: raise ValueError('Map projecction not supported')
    
    fig, axes = plt.subplots(2,2, 
                             subplot_kw=dict(projection=mapproj),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(20,15))
    fig.tight_layout()

    # 0-5 m
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('MarEDAT 0-5 m', fontsize=20)


    # 5-25 m
    m2 = axes[0,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('MarEDAT 5-25 m', fontsize=20)

    # 25-100 m
    m3 = axes[1,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('MarEDAT 25-100 m', fontsize=20)

    # 100-bottom m
    m4 = axes[1,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(100), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('MarEDAT 100 m - bottom', fontsize=20)

    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[1,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50'], fontsize=20) 
    cbar.set_label('Diatom Biomass [mg C m$^{-3}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

    plt.show()
    return fig

# Diatom carbon

# FESOM comparison to MarEDat

In [ ]:
def fesom_to_maredat_levels(resultpath, runid, years, mesh, lon_maredat, lat_maredat):
    '''
    Process FESOM data analogue to MarEDAT layers 0-5, 5-25, 25-100, 100-bottom
    as means
    and interpolate to 1x1 grid
    
    Input:
    resultpath as list
    runid as list
    years 
    mesh: mesh object as loaded with pyfesom2
    lon_maredat
    lat_maredat
    
    Output:
    DiaC_interp_all: array with FESOM on 1x1 grid, layered means
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
        
    # get interpolation input -> data of all levels
    # annual mean of fesom data already caluclated when reading model results
    print('\nProcessing {0} for years {1}-{2}'.format(resultpath,years[0],years[-1]))
    
    DiaC_interp_in = pf.get_data(resultpath, "DiaC", years, mesh, runid=runid, how="mean")
    DiaC_interp_in = DiaC_interp_in * 12.01
    #print(np.shape(DiaC_interp_in))

    # interp preperation
    lons, lats = np.meshgrid(lon_maredat, lat_maredat)

    # load FESOM mesh diag 
    diag = pf.get_meshdiag(mesh)
    #print(diag)
    #print(diag['Z']) # depth of layers
    mesh_depths = diag['Z'].values

    # depth indices similar to MarEDAT
    depth_range = [0,5,25,100,1e5]
    dep_ind1  = np.nonzero((0 >= mesh_depths )&(mesh_depths >= -5))
    dep_ind2  = np.nonzero((-5 > mesh_depths )&(mesh_depths >= -25))
    dep_ind3  = np.nonzero((-25 > mesh_depths )&(mesh_depths >= -100))
    dep_ind4  = np.nonzero((-100 > mesh_depths))

    dep_ind = [dep_ind1,dep_ind2,dep_ind3,dep_ind4]

    # storage
    DiaC_interp_all = [0]*len(dep_ind)

    for d in range(len(dep_ind)):
        d1 = depth_range[d]
        d2 = depth_range[d+1]
        dep_i = dep_ind[d]
        print('\nDepth range {0}-{1} m'.format(d1,d2))
        #print('chose indices:'), print(dep_i)

        DiaC_interp_dep = np.squeeze(DiaC_interp_in[:,dep_i], axis=1)
        DiaC_interp_dep = np.nanmean(DiaC_interp_dep,axis=1)
        #print(np.shape(DiaC_interp_dep))

        # interpolate fesom data to 1x1 grid
        DiaC_interp = pf.fesom2regular(
                data = DiaC_interp_dep,
                mesh = mesh,
                lons = lons, 
                lats = lats)

        #print(np.shape(DiaC_interp))
        print('with mean {0:6.4f} min {1:8.6f} max {2:6.4f}'.format(np.nanmean(DiaC_interp), np.nanmin(DiaC_interp), np.nanmax(DiaC_interp)))

        # save to storage
        DiaC_interp_all[d] = DiaC_interp
        
    return DiaC_interp_all

In [ ]:
### plot FESOM overview
def fesom_plot_depths(model, figlabel, savelabel, lon_maredat, lat_maredat, 
                      mapproj, years, savefig=False, savepath=''):
    '''
    Plot log10 FESOM layered means
    
    Input:
    model: FESOM data array with 4 data layers, i.e. DiaC_interp_all as output of fesom_to_maredat_levels()
    figlabel: used for xlabel
    savelabel: used to save the produced graph ('FESOM_depths_asMarEDat_'+label...)
    lon_maredat
    lat_maredat
    mapproj: as ccrs format
    savefig=False
    savepath
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    if(mapproj == 'pc'): mapproj = ccrs.PlateCarree()
    elif(mapproj == 'rob'): mapproj = ccrs.Robinson()
    else: raise ValueError('Map projecction not supported')   
        
    if(savepath == ''):
        if(savefig == True):
            raise ValueError('Input for saving graph insufficient')
        
    print(figlabel)
    
    fig, axes = plt.subplots(2,2, 
                             subplot_kw=dict(projection=mapproj),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(20,15))
    fig.tight_layout()

    # 0-5 m
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(model[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)


    # 5-25 m
    m2 = axes[0,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(model[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('FESOM 5-25 m', fontsize=20)

    # 25-100 m
    m3 = axes[1,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(model[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('FESOM 25-100 m', fontsize=20)

    # 100-bottom m
    m4 = axes[1,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(model[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('FESOM 100 m - bottom', fontsize=20)

    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[1,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50),np.log10(100),np.log10(200)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50','100','200'], fontsize=20) 
    cbar.set_label('Run '+figlabel+', Diatom Biomass [mg C m$^{-3}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

    if(savefig == True):
        fig.savefig(savepath+'FESOM_depths_asMarEDat_'+savelabel+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    plt.show()

# MarEDAT and FESOM side by side

In [ ]:
def plot_2cols(model, figlabel, savelabel, maredat_layered, lon_maredat, lat_maredat, 
                      mapproj, years, savefig=False, savepath=''):
    '''
    Plot MarEDAT and FESOM data side by side in two columns, seperated by depth ranges
    
    Input:
    model: FESOM data array with 4 data layers, i.e. DiaC_interp_all as output of fesom_to_maredat_levels()
    figlabel: part of xlabel for plot
    savelabel: used to save the produced graph ('MarEDAT_FESOM_depths_'+label...)
    maredat_layered: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom
    lon_maredat
    lat_maredat
    mapproj: as ccrs format 
    savefig=False
    savepath
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    if(mapproj == 'pc'): mapproj = ccrs.PlateCarree()
    elif(mapproj == 'rob'): mapproj = ccrs.Robinson()
    else: raise ValueError('Map projecction not supported')
        
    if(savepath == ''):
        if(savefig == True):
            raise ValueError('Input for saving graph insufficient')
    
    print(figlabel)
    
    fig, axes = plt.subplots(4,2, 
                             subplot_kw=dict(projection=mapproj),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(30,45))
    #fig.tight_layout()

    # 0-5 m maredat ---------------------------------------------------------------------------------------------
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('MarEDAT 0-5 m', fontsize=20)

    # 0-5 m FESOM
    m1 = axes[0,1]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(model[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)

    # 5-25 m maredat ---------------------------------------------------------------------------------------------
    m2 = axes[1,0]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('MarEDAT 5-25 m', fontsize=20)

    # 5-25 m FESOM
    m2 = axes[1,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(model[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('FESOM 5-25 m', fontsize=20)

    # 25-100 m maredat ---------------------------------------------------------------------------------------------
    m3 = axes[2,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('MarEDAT 25-100 m', fontsize=20)

    # 25-100 m FESOM
    m3 = axes[2,1]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(model[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('FESOM 25-100 m', fontsize=20)

    # 100-bottom m maredat ---------------------------------------------------------------------------------------------
    m4 = axes[3,0]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('MarEDAT 100 m - bottom', fontsize=20)

    # 100-bottom m
    m4 = axes[3,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(model[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('FESOM 100 m - bottom', fontsize=20)

    # ---------------------------------------------------------------------------------------------
    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[3,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50),np.log10(100)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50','100'], fontsize=20) 
    cbar.set_label('Diatom Biomass [mg C m$^{-3}$]\nRun '+figlabel, fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

    if(savefig == True):
        fig.savefig(savepath+'MarEDAT_FESOM_depths_'+savelabel+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    plt.show()

### apply mask

In [ ]:
# mask model where MarEDAT not available


def mask_model_with_maredat(model,maredat_layered):
    '''
    Mask model data where no MarEDAT available, then fill masked_array for better processing
    
    Input:
    model: FESOM data array with 4 data layers, i.e. DiaC_interp_all as output of fesom_to_maredat_levels()
    maredat_layered: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom
    
    '''
    
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pylab as plt
    import cartopy.crs as ccrs
    
    model_ma = [0]*4
    model_ma[0] = np.ma.filled(np.ma.where(np.isnan(maredat_layered[0]),np.nan, model[0]), fill_value=0)
    #model_ma[0] = np.ma.where(np.isnan(d_biomass_0_5),np.nan, model[0])

    model_ma[1] = np.ma.filled(np.ma.where(np.isnan(maredat_layered[1]),np.nan, model[1]), fill_value=0)
    model_ma[2] = np.ma.filled(np.ma.where(np.isnan(maredat_layered[2]),np.nan, model[2]), fill_value=0)
    model_ma[3] = np.ma.filled(np.ma.where(np.isnan(maredat_layered[3]),np.nan, model[3]), fill_value=0)
    
    return model_ma
    
    if False: # first manual masking
        mo5_ma = [0]*4
        mo5_ma[0] = np.ma.where(np.isnan(d_biomass_0_5),np.nan, mo5[0])
        mo5_ma[1] = np.ma.where(np.isnan(d_biomass_5_25),np.nan, mo5[1])
        mo5_ma[2] = np.ma.where(np.isnan(d_biomass_25_100),np.nan, mo5[2])
        mo5_ma[3] = np.ma.where(np.isnan(d_biomass_100_bot),np.nan, mo5[3])
    

# Taylor statistics

In [ ]:
def plot_Taylor(ref,pred,plot_depth,plot=True):
    '''
    Plot Taylor diagram for one depth layer mean
    
    Input:
    ref: reference/observation data
    pred: predicted/model data
    plot_depth: str to print label
    '''
    
    import skill_metrics as sm
    from plot_Taylor_normalized import plt_Taylor_norm

    title = 'At {0} m'.format(plot_depth)
    fig, sdev, crmsd, ccoef = plt_Taylor_norm(ref,pred,mask=True,title=title,plot=plot)
    # applied mask in function is:
    # ind_stat = np.where(data_pred != 0)

    return fig, sdev, crmsd, ccoef

In [ ]:
def plot_Taylor_all_depths(ref,pred,depth_range,plot=True):
    '''
    on-top function for plot_Taylor()
    
    Input: 
    ref,pred: array with 4 depth layers for reference/obs and pred/model data
    see also docstring of plot_Taylor?
    depth_range: used for plot title
    
    Output: Taylor statistics and diagram for 4 depth layers
    fig, sdev, crmsd, ccoef
    '''

    from F2R_sensitivity_toolbox import plot_Taylor
    
    # create storage arrays
    fig = [0]*4
    sdev = [0]*4
    crmsd = [0]*4
    ccoef = [0]*4
    
    for d in range(4):
        d1 = depth_range[d]
        d2 = depth_range[d+1]
        print('\nDepth range {0}-{1} m'.format(d1,d2))

        fig[d], sdev[d], crmsd[d], ccoef[d] = plot_Taylor(ref[d],pred[d], plot_depth = '{0}-{1}'.format(d1,d2),plot=plot)
    
    return fig, sdev, crmsd, ccoef

# Scatter plots (as Miriam)

In [ ]:
### define function
def plot_scatter(model_masked, figlabel, savelabel, maredat_layered, 
                years, savefig=False, savepath='',
                corr = False):
    '''
    Plot MarEDAT ~ FESOM scattered in two columns (overview & focus), 
    seperated by depth ranges
    
    Input:
    model_masked: FESOM data array with 4 data layers
    figlabel: used for xlabel
    savelabel: used to save the produced graph ('MarEDAT_scatter_'+savelabel...)
    maredat_layered: maredat data as array[4] layered into 0-5, 5-25, 25-100, 100m-bottom
    savefig=False
    savepath
    corr: array(4,2) containing correlation coefficients (out of plot_Taylor)
    '''
    
    import numpy as np
    import matplotlib.pylab as plt
        
    if(savepath == ''):
        if(savefig == True):
            raise ValueError('Input for saving graph insufficient')
            
    if(corr != False):
        corr0 = corr[0][1]
        corr1 = corr[1][1]
        corr2 = corr[2][1]
        corr3 = corr[3][1]
    
    print(figlabel)
    
    fig, axes = plt.subplots(4,2, #gridspec_kw={'hspace': 0.001, 'wspace': 0.1}, 
                             figsize=(10,10))

    m1 = axes[0,0]
    m1.plot(np.log10(model_masked[0]),np.log10(maredat_layered[0]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 0-5 m, ccoef = {0:5.4f}".format(corr0),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 0-5 m",color='darkred')
    

    m1 = axes[0,1]
    m1.plot(np.log10(model_masked[0]),np.log10(maredat_layered[0]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(1,2)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(1.1,4, "Depth range: 0-5 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[1,0]
    m1.plot(np.log10(model_masked[1]),np.log10(maredat_layered[1]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 5-25 m, ccoef = {0:5.4f}".format(corr1),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 5-25 m",color='darkred')

    m1 = axes[1,1]
    m1.plot(np.log10(model_masked[1]),np.log10(maredat_layered[1]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(1,2)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(1.1,4, "Depth range: 5-25 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[2,0]
    m1.plot(np.log10(model_masked[2]),np.log10(maredat_layered[2]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 25-100 m, ccoef = {0:5.4f}".format(corr2),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 25-100 m",color='darkred')

    m1 = axes[2,1]
    m1.plot(np.log10(model_masked[2]),np.log10(maredat_layered[2]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(0,1.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(0.15,4, "Depth range: 25-100 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[3,0]
    m1.plot(np.log10(model_masked[3]),np.log10(maredat_layered[3]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.set_xlabel('Model '+figlabel+' Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    if(corr != False):
        m1.text(-2.5,4, "Depth range: 100 m - bot, ccoef = {0:5.4f}".format(corr3),color='darkred')
    else: m1.text(-2.5,4, "Depth range: 100 m - bot",color='darkred')

    m1 = axes[3,1]
    m1.plot(np.log10(model_masked[3]),np.log10(maredat_layered[3]),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-2,0.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.set_xlabel('Model '+figlabel+' Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(-1.7,4, "Depth range: 100 m - bottom",color='darkred')
    
    if(savefig == True):
        fig.savefig(savepath+'MarEDAT_scatter_'+savelabel+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
    plt.show()

# OLD STUFF

In [ ]:
### get interpolation input -> data of all levels
if False:
    # annual mean of fesom data already caluclated when reading model results
    i=0
    DiaC_interp_in = pf.get_data(resultpath[i], "DiaC", years, mesh, runid=idlist[i], how="mean")
    DiaC_interp_in = DiaC_interp_in * 12.01
    print(np.shape(DiaC_interp_in))

    # interp preperation
    lons, lats = np.meshgrid(lon_maredat, lat_maredat)

    # depth indices similar to MarEDAT
    depth_range = [0,5,25,100,1e5]
    dep_ind1  = np.nonzero((0 >= mesh_depths )&(mesh_depths >= -5))
    dep_ind2  = np.nonzero((-5 > mesh_depths )&(mesh_depths >= -25))
    dep_ind3  = np.nonzero((-25 > mesh_depths )&(mesh_depths >= -100))
    dep_ind4  = np.nonzero((-100 > mesh_depths))

    dep_ind = [dep_ind1,dep_ind2,dep_ind3,dep_ind4]

    #print(dep_ind1, dep_ind4)

    # storage
    DiaC_interp_all = [0]*len(idlist)

    for d in range(len(dep_ind)):
        d1 = depth_range[d]
        d2 = depth_range[d+1]
        dep_i = dep_ind[d]
        print('\nProcessing {4}\nDepth range {0}-{1} m for years {2}-{3}'.format(d1,d2,years[0],years[-1],resultpath[i]))

        DiaC_interp_dep = np.squeeze(DiaC_interp_in[:,dep_ind1])
        #print(np.shape(DiaC_interp_dep))

        # interpolate fesom data to 1x1 grid
        DiaC_interp = pf.fesom2regular(
                data = DiaC_interp_dep,
                mesh = mesh,
                lons = lons, 
                lats = lats)

        #print(np.shape(DiaC_interp))

        # save to storage
        DiaC_interp_all[i] = DiaC_interp


In [ ]:
#### manually calculating and plotting Taylor statistics
if False:
    # get indices for valid data points 
    ind_stat = np.where(mo5_ma[2] != 0)

    taylor_stats1 = sm.taylor_statistics(mo5_ma[2][ind_stat],d_biomass_25_100[ind_stat])
    print(taylor_stats1)

    sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])

    crmsd = np.array([taylor_stats1['crmsd'][0]/taylor_stats1['sdev'][0],
                      taylor_stats1['crmsd'][1]/taylor_stats1['sdev'][0]])

    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    print('\nNormalized Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
                sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))

    fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
    sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                          titleOBS = 'observation', markerLabel = ['obs','model'],
                          markerLabelColor = 'c',
                          markerColor = 'c', markerLegend = 'on',
                          markerSize = 12,
                          #tickRMS = range(0,5,1),
                          #tickRMSangle = tickRMSangle,
                          colRMS = 'm', styleRMS = ':', #widthRMS = 2.0,
                          titleRMS = 'off', #tickSTD = np.arange(0,1.25,.25),
                          #axismax = 1.25, 
                          colSTD = 'b', styleSTD = '-.',
                          widthSTD = 1.0, titleSTD = 'on',
                          colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                          titleCOR = 'on')

In [ ]:
### old scatter plot, manually
if False:
    fig, axes = plt.subplots(4,2, #gridspec_kw={'hspace': 0.001, 'wspace': 0.1}, 
                             figsize=(10,10))

    m1 = axes[0,0]
    m1.plot(np.log10(mo5_ma[0]),np.log10(d_biomass_0_5),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(-2.5,4, "Depth range: 0-5 m",color='darkred')

    m1 = axes[0,1]
    m1.plot(np.log10(mo5_ma[0]),np.log10(d_biomass_0_5),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(1,2)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(1.1,4, "Depth range: 0-5 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[1,0]
    m1.plot(np.log10(mo5_ma[1]),np.log10(d_biomass_5_25),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(-2.5,4, "Depth range: 5-25 m",color='darkred')

    m1 = axes[1,1]
    m1.plot(np.log10(mo5_ma[1]),np.log10(d_biomass_5_25),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(1,2)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(1.4,4, "Depth range: 5-25 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[2,0]
    m1.plot(np.log10(mo5_ma[2]),np.log10(d_biomass_25_100),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(-2.5,4, "Depth range: 25-100 m",color='darkred')

    m1 = axes[2,1]
    m1.plot(np.log10(mo5_ma[2]),np.log10(d_biomass_25_100),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(0,1.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(0.2,4, "Depth range: 25-100 m",color='darkred')

    # ---------------------------------------------------------------------------------------------------

    m1 = axes[3,0]
    m1.plot(np.log10(mo5_ma[3]),np.log10(d_biomass_100_bot),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-3,2.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.set_xlabel('Model "mo5" Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(-2.5,4, "Depth range: 100 m - bottom",color='darkred')

    m1 = axes[3,1]
    m1.plot(np.log10(mo5_ma[3]),np.log10(d_biomass_100_bot),'.',color='black')
    m1.plot([-4,4], [-4,4])
    m1.set_xlim(-2,0.5)
    m1.set_ylim(-5,5)
    #plt.yticks([-5,-4,-3,-2,-1,0,1,2,3])
    #m1.set_ylabel('Maredat Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.set_xlabel('Model "mo5" Log$_{10}$ biomass \n [mg C m$^{-3}$]')
    m1.text(-1.5,4, "Depth range: 100 m - bottom",color='darkred')

    fig.savefig(savepath)

In [ ]:
if False:
    ### plot FESOM overview
    fig, axes = plt.subplots(2,2, 
                             subplot_kw=dict(projection=ccrs.Robinson()),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(20,15))
    fig.tight_layout()

    # 0-5 m
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)


    # 5-25 m
    m2 = axes[0,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('FESOM 5-25 m', fontsize=20)

    # 25-100 m
    m3 = axes[1,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('FESOM 25-100 m', fontsize=20)

    # 100-bottom m
    m4 = axes[1,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('FESOM 100 m - bottom', fontsize=20)

    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[1,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50),np.log10(100),np.log10(200)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50','100','200'], fontsize=20) 
    cbar.set_label('Run "mo5", Diatom Biomass [mg C m$^{-3}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)



In [ ]:
### maredat fesom side by side
if False:
    fig, axes = plt.subplots(4,2, 
                             subplot_kw=dict(projection=ccrs.Robinson()),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(30,45))
    fig.tight_layout()

    # 0-5 m maredat ---------------------------------------------------------------------------------------------
    m1 = axes[0,0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(d_biomass_0_5), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('MarEDAT 0-5 m', fontsize=20)

    # 0-5 m FESOM
    m1 = axes[0,1]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)

    # 5-25 m maredat ---------------------------------------------------------------------------------------------
    m2 = axes[1,0]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(d_biomass_5_25), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('MarEDAT 5-25 m', fontsize=20)

    # 5-25 m FESOM
    m2 = axes[1,1]
    f2 = m2.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[1]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m2)
    m2.set_title('FESOM 5-25 m', fontsize=20)

    # 25-100 m maredat ---------------------------------------------------------------------------------------------
    m3 = axes[2,0]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(d_biomass_25_100), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('MarEDAT 25-100 m', fontsize=20)

    # 25-100 m FESOM
    m3 = axes[2,1]
    f3 = m3.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[2]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m3)
    m3.set_title('FESOM 25-100 m', fontsize=20)

    # 100-bottom m maredat ---------------------------------------------------------------------------------------------
    m4 = axes[3,0]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(d_biomass_100_bot), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('MarEDAT 100 m - bottom', fontsize=20)

    # 100-bottom m
    m4 = axes[3,1]
    f4 = m4.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5[3]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    mygrid(m4)
    m4.set_title('FESOM 100 m - bottom', fontsize=20)

    # ---------------------------------------------------------------------------------------------
    # add one colorbar for all plots below figure
    cbar = fig.colorbar(f1,
                        ax = axes[3,:2], 
                        location ='bottom',
                        ticks=[np.log10(0.001),np.log10(0.01), np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(10),np.log10(50),np.log10(100)],
                        fraction=0.1, pad=0.1) 
    cbar.ax.set_xticklabels(['0.001','0.01','0.1','0.5','1','10','50','100'], fontsize=20) 
    cbar.set_label('Diatom Biomass [mg C m$^{-3}$]', fontsize=20)
    #cbar.set_label('Log$_{10}$ Biomass [$\mu$g L $^{-1}$]', fontsize=20)

